Nuevamente, vamos a leer primero unos datos...

In [1]:
# primero hacemos los imports de turno
import os
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

# Lectura de un fichero de datos

In [2]:
# Leemos los datos del fichero 'mast.txt'
ipath = os.path.join('Datos', 'mast.txt')

def dateparse(date, time):
    YY = 2000 + int(date[:2])
    MM = int(date[2:4])
    DD = int(date[4:])
    hh = int(time[:2])
    mm = int(time[2:])
    
    return dt.datetime(YY, MM, DD, hh, mm, 0)
    

cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols, 
                   parse_dates = {'Timestamp': [0, 1]}, index_col = 0,
                   date_parser = dateparse)

d:\users\X003621\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\io\parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


# Lectura de un segundo fichero de datos

In [3]:
# Leemos los datos del fichero 'model.txt'
ipath = os.path.join('Datos', 'model.txt')

model = pd.read_csv(ipath, sep = "\s*", skiprows = 3,
                    parse_dates = {'Timestamp': [0, 1]}, index_col = 'Timestamp')

d:\users\X003621\AppData\Local\Continuum\Miniconda3\lib\site-packages\pandas\io\parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [4]:
for c in ['x1','x2','x3','x4','x5']: # Eliminamos unas columnas innecesarias
    _ = wind.pop(c)
wind.head(3)

,wspd,wspd_max,wdir,wspd_std
Timestamp,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,0.11
2013-09-04 00:10:00,1.69,2.31,99.9,0.35
2013-09-04 00:20:00,1.28,1.50,96.0,0.08


In [5]:
model.head(3)

,M(m/s),D(deg),T(C),De(k/m3),PRE(hPa),RiNumber,RH(%)
Timestamp,,,,,,,
1984-01-01 00:00:00,20.8,243,7.3,1.25,1002.8,0.04,86.3
1984-01-01 01:00:00,20.8,243,7.3,1.25,1002.2,0.04,86.9
1984-01-01 02:00:00,20.6,243,7.4,1.24,1001.8,0.04,87.0


In [6]:
wind['Timestamp'] = wind.index
print(wind['Timestamp'].diff().min())
del wind['Timestamp']

0 days 00:10:00


In [7]:
model['Timestamp'] = model.index
print(model['Timestamp'].diff().min())
del model['Timestamp']

0 days 01:00:00


Tenemos datos con una frecuencia temporal mínima de 10 minutos (`wind`) frente a unos segundos datos con una frecuencia temporal de 1 hora (`model`).

# Inciso: `axis` 101

En muchas ocasiones vamos a encontrar una *keyword* llamada `axis`. Veamos en un momento cómo funciona en pandas para evitarnos posibles problemas e incongruencias:

## Posibilidades

* axis = 0 (actúa sobre las filas/*rows*)
* axis = 1 (actúa sobre las columnas/*columns*)
* <span style="color:#888">axis = 2 (solo para `Panel`)</span>

![](imgs/DF_Rows_Columns.jpg)
(fuente: http://stackoverflow.com/a/25774395/5216568).

<br>
<div class="alert alert-info">
<p><b>Regla nemotécnica:</b></p> 
<p>Puedes pensar que el '1' es como una columna.</p>
<p><b>Otras opciones:</b></p> 
<p>Otra opción sería usar `axis = 'index'` (similar a `axis = 0`) o `axis = 'columns'` (similar a `axis = 1`) para `DataFrame`s. Para `Panel`es tendríamos `items`, `minor`, `major` (similar a las opciones 0, 1 o 2).</p>.
<p>Para `DataFrame`s también podéis usar `index = 'rows'`, que me parece que es más evidente que `'index'` pero no lo recomiendo ya que no está documentado en ningún sitio.</p>
<p>Además, usar `'index'`, `'rows'`, `'columns'`,..., puede llegar a ser confuso ya que en muchos sitios nos encontraremos con *keywords* que usan esa nomenclatura.</p>.
</div>

Pero, que significa que 'actúa sobre las filas/columnas'. Veamos algunos ejemplo simples para ver si nos queda un poco más claro:

In [8]:
df = pd.DataFrame(np.array([[1, 10], [2, 20], [3,30]]), columns = ['A', 'B'])
df

,A,B
0,1,10
1,2,20
2,3,30


Si no indicamos nada, por defecto, las operaciones se realizan sobre las filas (`axis = 0`), es decir, se cogen todos los elementos de fila de cada columna:

In [9]:
df.sum()

A     6
B    60
dtype: int64

In [10]:
# Lo anterior sería similar a 
df.sum(axis = 0)

A     6
B    60
dtype: int64

Si queremos que nos dé el resultado de cada fila, es decir, se cogen todos los elementos de columna de una fila, debemos indicar que `axis = 1`:

In [11]:
df.sum(axis = 1)

0    11
1    22
2    33
dtype: int64

Otro ejemplo:

In [12]:
df < 10

,A,B
0,True,False
1,True,False
2,True,False


In [13]:
(df < 10).all()

A     True
B    False
dtype: bool

In [14]:
(df < 10).all(axis = 'columns') # en lugar de axis = 1 usamos axis = 'columns'

0    False
1    False
2    False
dtype: bool

In [15]:
# Probad operaciones sobre df usando axis = 0, 1, 'index', rows', columns'


Espero que haya quedado un poco claro con estos ejemplos simples.

# Uniendo estructuras de datos pandas

Lo que [vamos a ver no es evidente](http://pandas.pydata.org/pandas-docs/stable/merging.html) y, en algunos casos, es conveniente conocer [algebral relacional](https://en.wikipedia.org/wiki/Relational_algebra) para poder enterder qué es lo que está pasando.

## Uniendo datos usando `concat`

In [16]:
new = pd.concat([wind, model], axis = 0, join = 'outer')

In [17]:
new.head(5)

,D(deg),De(k/m3),M(m/s),PRE(hPa),RH(%),RiNumber,T(C),wdir,wspd,wspd_max,wspd_std
Timestamp,,,,,,,,,,,
2013-09-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.5,2.21,2.58,0.11
2013-09-04 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.9,1.69,2.31,0.35
2013-09-04 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,1.28,1.50,0.08
2013-09-04 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.2,1.94,2.39,0.26
2013-09-04 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.4,2.17,2.67,0.23


In [18]:
new.tail(5)

,D(deg),De(k/m3),M(m/s),PRE(hPa),RH(%),RiNumber,T(C),wdir,wspd,wspd_max,wspd_std
Timestamp,,,,,,,,,,,
2015-09-10 19:00:00,85,1.22,9.2,1008.5,67.9,0.25,16.1,NaN,NaN,NaN,NaN
2015-09-10 20:00:00,85,1.21,9.2,1008.3,68.0,0.28,16.2,NaN,NaN,NaN,NaN
2015-09-10 21:00:00,87,1.21,9.5,1008.1,69.1,0.24,16.2,NaN,NaN,NaN,NaN
2015-09-10 22:00:00,90,1.21,10.0,1008.0,72.3,0.10,16.0,NaN,NaN,NaN,NaN
2015-09-10 23:00:00,94,1.22,10.0,1007.9,73.9,-0.06,15.8,NaN,NaN,NaN,NaN


In [19]:
new.loc['2014/01/01 00:00':'2014/01/01 02:00']

,D(deg),De(k/m3),M(m/s),PRE(hPa),RH(%),RiNumber,T(C),wdir,wspd,wspd_max,wspd_std
Timestamp,,,,,,,,,,,
2014-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.6,12.15,13.95,0.60
2014-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.5,11.83,13.21,0.63
2014-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.3,11.05,13.43,0.67
2014-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219.9,11.84,13.59,0.60
2014-01-01 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218.9,12.99,15.43,0.89
2014-01-01 00:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219.6,12.89,16.02,1.00
2014-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217.4,13.39,15.05,0.71
2014-01-01 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215.6,13.23,15.62,0.88
2014-01-01 01:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.6,13.69,14.93,0.55


![](imgs/merging_concat_basic.png)

In [20]:
new = pd.concat([wind, model], axis = 1, join = 'inner')

In [21]:
new.head(5)

,wspd,wspd_max,wdir,wspd_std,M(m/s),D(deg),T(C),De(k/m3),PRE(hPa),RiNumber,RH(%)
Timestamp,,,,,,,,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,0.11,3.8,118,17.3,1.22,1013.4,1.87,91.0
2013-09-04 01:00:00,2.66,2.96,114.4,0.12,3.9,112,17.1,1.22,1013.2,1.03,94.4
2013-09-04 02:00:00,4.70,5.00,120.8,0.12,4.0,114,17.0,1.22,1012.9,0.65,96.1
2013-09-04 03:00:00,4.72,5.17,124.5,0.16,4.2,116,17.0,1.22,1012.7,0.55,95.8
2013-09-04 04:00:00,3.50,3.85,123.3,0.15,4.2,120,16.9,1.22,1012.4,0.53,95.0


In [22]:
new.loc['2014/01/01 00:00':'2014/01/01 02:00']

,wspd,wspd_max,wdir,wspd_std,M(m/s),D(deg),T(C),De(k/m3),PRE(hPa),RiNumber,RH(%)
Timestamp,,,,,,,,,,,
2014-01-01 00:00:00,12.15,13.95,224.6,0.60,12.7,206,7.4,1.23,992.4,0.03,90.9
2014-01-01 01:00:00,13.39,15.05,217.4,0.71,12.7,206,7.4,1.23,992.4,0.03,89.2
2014-01-01 02:00:00,15.81,17.99,210.9,0.81,13.1,202,7.4,1.23,992.3,0.02,87.8


`concat` permite 'unir' estructuras de datos pandas usando filas o columnas. 

Y lo anterior no os ha quedado nada claro!!! Y no habéis preguntado!!!

Veamos un ejemplo más simple:

In [23]:
np.random.seed(123)
df1 = pd.DataFrame(np.random.randn(10,2), 
                   columns = ['A', 'B'], 
                   index = np.arange(10))
df2 = pd.DataFrame(np.random.randn(4,3), 
                   columns = ['A', 'B', 'C'], 
                   index = np.arange(8, 12))

In [24]:
df1

,A,B
0,-1.085631,0.997345
1,0.282978,-1.506295
2,-0.578600,1.651437
3,-2.426679,-0.428913
4,1.265936,-0.866740
5,-0.678886,-0.094709
6,1.491390,-0.638902
7,-0.443982,-0.434351
8,2.205930,2.186786
9,1.004054,0.386186


In [25]:
df2

,A,B,C
8,0.737369,1.490732,-0.935834
9,1.175829,-1.253881,-0.637752
10,0.907105,-1.428681,-0.140069
11,-0.861755,-0.255619,-2.798589


In [26]:
new = pd.concat([df1, df2], axis = 0, join = 'inner')
new

,A,B
0,-1.085631,0.997345
1,0.282978,-1.506295
2,-0.578600,1.651437
3,-2.426679,-0.428913
4,1.265936,-0.866740
5,-0.678886,-0.094709
6,1.491390,-0.638902
7,-0.443982,-0.434351
8,2.205930,2.186786
9,1.004054,0.386186


In [27]:
new = pd.concat([df1, df2], axis = 1, join = 'inner')
new

,A,B,A,B,C
8,2.205930,2.186786,0.737369,1.490732,-0.935834
9,1.004054,0.386186,1.175829,-1.253881,-0.637752


Normalmente uso esta última opción con nombres de columnas diferentes porque normalmente es lo que quiero hacer...

## Concatenando usando el método `append`

Podemos hacer algo parecido a lo anterior usando el método `append` de las estructuras de datos:

In [28]:
wind.append(model)

,D(deg),De(k/m3),M(m/s),PRE(hPa),RH(%),RiNumber,T(C),wdir,wspd,wspd_max,wspd_std
Timestamp,,,,,,,,,,,
2013-09-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.5,2.21,2.58,0.11
2013-09-04 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.9,1.69,2.31,0.35
2013-09-04 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,1.28,1.50,0.08
2013-09-04 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.2,1.94,2.39,0.26
2013-09-04 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.4,2.17,2.67,0.23
2013-09-04 00:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,2.25,2.89,0.35
2013-09-04 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.4,2.66,2.96,0.12
2013-09-04 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.1,3.55,4.33,0.46
2013-09-04 01:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.1,4.26,4.50,0.08


Normalmente esto no es lo que quiero hacer. Normalmente quiero juntar con cierta lógica estructuras de datos pandas y para ello podemos usar `pd.merge`...

## Usando `pd.merge` como en una base de datos SQL

In [29]:
pd.merge(wind, model, left_index = True, right_index = True, how = 'inner').head(5)

,wspd,wspd_max,wdir,wspd_std,M(m/s),D(deg),T(C),De(k/m3),PRE(hPa),RiNumber,RH(%)
Timestamp,,,,,,,,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,0.11,3.8,118,17.3,1.22,1013.4,1.87,91.0
2013-09-04 01:00:00,2.66,2.96,114.4,0.12,3.9,112,17.1,1.22,1013.2,1.03,94.4
2013-09-04 02:00:00,4.70,5.00,120.8,0.12,4.0,114,17.0,1.22,1012.9,0.65,96.1
2013-09-04 03:00:00,4.72,5.17,124.5,0.16,4.2,116,17.0,1.22,1012.7,0.55,95.8
2013-09-04 04:00:00,3.50,3.85,123.3,0.15,4.2,120,16.9,1.22,1012.4,0.53,95.0


In [30]:
(pd.merge(wind, model, left_index = True, right_index = True, how = 'inner') == 
 pd.concat([wind, model], axis = 1, join = 'inner')).all().all()

True

Imaginemos que queremos unir dos `DataFrame`s usando valores de columnas:

In [39]:
df1 = pd.DataFrame(
    np.array([
        np.arange(1, 11),
        np.random.choice([1,2,3], size = 10),
        np.arange(1, 11) * 10
    ]).T,
    columns = ['A', 'col', 'B']
)
df2 = pd.DataFrame(
    np.array([
        np.arange(11, 21),
        np.random.choice([1,2,3], size = 10),
        np.arange(1, 11) * 100
    ]).T,
    columns = ['A', 'col', 'B']
)
display(df1)
display(df2)

,A,col,B
0,1,3,10
1,2,3,20
2,3,1,30
3,4,1,40
4,5,2,50
5,6,3,60
6,7,3,70
7,8,3,80
8,9,1,90
9,10,1,100


,A,col,B
0,11,1,100
1,12,3,200
2,13,2,300
3,14,3,400
4,15,3,500
5,16,3,600
6,17,3,700
7,18,1,800
8,19,3,900
9,20,1,1000


In [40]:
pd.merge(df1, df2, on = ['col'])

,A_x,col,B_x,A_y,B_y
0,1,3,10,12,200
1,1,3,10,14,400
2,1,3,10,15,500
3,1,3,10,16,600
4,1,3,10,17,700
5,1,3,10,19,900
6,2,3,20,12,200
7,2,3,20,14,400
8,2,3,20,15,500
9,2,3,20,16,600


In [41]:
# Jugad un poco y mirad las keywords del pd.merge para ver otras opciones


## Combinando usando el método `join`

Un poco más de lo mismo. El método `join` nos ayuda, nuevamente, a unir estructuras de datos pandas. Vamos a ver unos pocos ejemplos rápidos:

In [43]:
wind.join(model, how = 'left').head(10)

ValueError: columns overlap but no suffix specified: Index(['col'], dtype='object')

In [44]:
wind.head()

,wspd,wspd_max,wdir,wspd_std,col
Timestamp,,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,0.11,2
2013-09-04 00:10:00,1.69,2.31,99.9,0.35,3
2013-09-04 00:20:00,1.28,1.50,96.0,0.08,1
2013-09-04 00:30:00,1.94,2.39,99.2,0.26,2
2013-09-04 00:40:00,2.17,2.67,108.4,0.23,1
